In [1]:
import torchvision.transforms as transforms

In [2]:
from auto_deep_learning.augmentation import ImageAugmentation

ImportError: cannot import name 'ImageAugmentation' from 'auto_deep_learning.augmentation' (/home/nil/.local/lib/python3.10/site-packages/auto_deep_learning/augmentation.py)

In [7]:
import auto_deep_learning as adl

In [3]:
from auto_deep_learning.utils import Loader

In [4]:
from auto_deep_learning.utils import ImageTransformer

In [11]:
new_img = ImageTransformer('1', (1, 2), {'1': '2'})

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [4]:
!pip uninstall auto_deep_learning -y

Found existing installation: auto-deep-learning 0.1.4.2
Uninstalling auto-deep-learning-0.1.4.2:
  Successfully uninstalled auto-deep-learning-0.1.4.2


In [2]:
!pip install auto_deep_learning==0.1.4.5

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: auto_deep_learning
    Found existing installation: auto-deep-learning 0.1.4.4
    Uninstalling auto-deep-learning-0.1.4.4:
      Successfully uninstalled auto-deep-learning-0.1.4.4
